In [1]:
# executeme

using NBInclude
@nbinclude("Hofstadter Single Particle in Julia.ipynb")

Hofstadter_SP (generic function with 1 method)

In [2]:
# executeme

#@time begin
using QuantumOptics

#https://juliapackages.com/p/einsum
using Einsum

# https://github.com/Jutho/TensorOperations.jl
using TensorOperations

using BenchmarkTools

using LinearAlgebra

#end

1) Hofstadter SP QoJulia energies are has to be equal to Hofstadter Single Particle energies.

In [3]:
Nx = 3; Ny = 3; N=Nx*Ny; q = 3; N_cut = q
PN = 2
U = 2

2

In [4]:
# executeme

function Hofstadter_SP_Op(Nx, Ny, alpha, Basis)
    
    H_SP = Hofstadter_SP(Nx,Ny,alpha, 0)
    
    N = Nx*Ny
    
    H = SparseOperator(Basis)

    for m in 1:N
        for n in 1:N
            H = H + H_SP[m,n] * transition(Basis, m, n)
        end
    end
    
    return H
end

Hofstadter_SP_Op (generic function with 1 method)

In [5]:
basis = NLevelBasis(N)

NLevel(N=9)

In [6]:
H1 = Hofstadter_SP_Op(Nx, Ny, 1/q, basis)

Operator(dim=9x9)
  basis: NLevel(N=9)sparse([2, 3, 4, 7, 1, 3, 5, 8, 1, 2  …  8, 9, 2, 5, 7, 9, 3, 6, 7, 8], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9], ComplexF64[-1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, -1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.4999999999999998 + 0.8660254037844387im, -1.0 + 0.0im, -1.0 + 0.0im  …  -1.0 + 0.0im, -1.0 + 0.0im, 0.4999999999999998 - 0.8660254037844387im, 0.4999999999999998 + 0.8660254037844387im, -1.0 + 0.0im, -1.0 + 0.0im, 0.5000000000000004 + 0.8660254037844386im, 0.5000000000000004 - 0.8660254037844386im, -1.0 + 0.0im, -1.0 + 0.0im], 9, 9)

In [7]:
eigenenergies(dense(H1))

9-element Vector{Float64}:
 -2.7320508075688794
 -2.732050807568878
 -2.7320508075688763
  0.7320508075688769
  0.732050807568877
  0.7320508075688779
  2.0000000000000004
  2.0000000000000004
  2.0000000000000018

In [8]:
eigenenergies(dense(H1)) == eigvals(Hofstadter_SP(Nx,Ny,1/q,0))

true

In [9]:
# executeme

function get_sub_states(SP_Op, N_cut)
    
    E0, states0 = eigenstates(dense(SP_Op))
    states = states0[1:N_cut]
    
    return states
end

get_sub_states (generic function with 1 method)

In [10]:
sub_states = get_sub_states(H1, N_cut);

In [11]:
# executeme

function get_Projector(states, Basis)
    
    b_sub = SubspaceBasis(Basis,states)
    P = projector(b_sub, Basis)
    Pt = dagger(P)
    
    return b_sub, P, Pt
end

get_Projector (generic function with 1 method)

In [12]:
b_sub, P, Pt = get_Projector(sub_states, basis);

In [13]:
# executeme

function Subspace_Op(SP_Op, P, Pt)
    return P*SP_Op*Pt
end

Subspace_Op (generic function with 1 method)

In [14]:
H1_sub = Subspace_Op(H1, P, Pt)

Operator(dim=3x3)
  basis: Subspace(superbasis=NLevel(N=9), states:3)
 -2.73205-0.0im           0.0+0.0im      -2.2e-16+1.1e-16im
 -1.1e-16+1.0e-17im  -2.73205-2.0e-17im  -1.0e-17+7.0e-17im
 -2.2e-16-1.0e-16im   3.0e-17+5.0e-17im  -2.73205+2.2e-16im

In [15]:
# executeme

function get_num_sub_list(basis,P,Pt)
    num_sub_list = []
    for m in 1:N
        NM = transition(basis, m, m)
        NMP = Subspace_Op(NM, P,Pt)
        push!(num_sub_list, NMP)
    end
    return num_sub_list
end

get_num_sub_list (generic function with 1 method)

In [16]:
num_sub_list = get_num_sub_list(basis,P,Pt);

In [17]:
states_mb = bosonstates(b_sub, PN) 
basis_mb = ManyBodyBasis(b_sub, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)

In [18]:
# executeme

function get_MB_Op(MB_Basis, Basis, Op)
    
    Op_MB = SparseOperator(MB_Basis)
    for i in 1:length(Basis)
        for j in 1:length(Basis)
            Op_MB = Op_MB + Op.data[i,j] * transition(MB_Basis, i, j)
        end
    end
    
    return Op_MB
end

get_MB_Op (generic function with 1 method)

In [19]:
H1_MB = get_MB_Op(basis_mb, b_sub, H1_sub)

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)sparse([1, 2, 3, 2, 3, 4, 5, 1, 2, 3  …  6, 4, 5, 2, 4, 5, 6, 3, 5, 6], [1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 6, 6, 6], ComplexF64[-5.464101615137758 + 0.0im, -1.6e-16 + 1.0e-17im, -3.1e-16 - 1.4e-16im, -5.464101615137757 - 2.0e-17im, 3.0e-17 + 5.0e-17im, -1.6e-16 + 1.0e-17im, -2.2e-16 - 1.0e-16im, -3.1e-16 + 1.6e-16im, -1.0e-17 + 7.0e-17im, -5.464101615137756 + 2.2e-16im  …  -3.1e-16 - 1.4e-16im, -5.464101615137757 - 4.0e-17im, 5.0e-17 + 7.0e-17im, -2.2e-16 + 1.1e-16im, -1.0e-17 + 1.0e-16im, -5.464101615137755 + 2.0e-16im, 5.0e-17 + 7.0e-17im, -3.1e-16 + 1.6e-16im, -1.0e-17 + 1.0e-16im, -5.464101615137756 + 4.4e-16im], 6, 6)

In [20]:
# H1_mb = manybodyoperator(basis_mb, H1_sub)

In [21]:
# executeme

function get_num_mb_list(basis_mb, b_sub, num_sub_list)
    num_mb_list = []
    for m in 1:N
        NMP = get_MB_Op(basis_mb, b_sub, num_sub_list[m])
        push!(num_mb_list, NMP)
    end
    return num_mb_list
end

get_num_mb_list (generic function with 1 method)

In [22]:
num_mb_list = get_num_mb_list(basis_mb, b_sub, num_sub_list);

In [23]:
function Int_Op(MB_Num_Op_List, MB_Basis, U)

    IT = SparseOperator(MB_Basis)
    
    for m in 1:N
        IT = IT + U/2 * ( MB_Num_Op_List[m]*MB_Num_Op_List[m] - MB_Num_Op_List[m] )
    end
    
    return IT
end

H_Int_MB = Int_Op(num_mb_list, basis_mb, U)

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)sparse([1, 2, 3, 4, 5, 6, 1, 2, 3, 4  …  3, 4, 5, 6, 1, 2, 3, 4, 5, 6], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6], ComplexF64[-1.0075951320351497 + 0.0im, -0.02600487162459121 - 0.00677931948613959im, 0.0861412154450642 - 0.04034373581488656im, 0.11816805340776146 - 0.01090490889120979im, 0.01228969051978185 - 0.002274276759884im, -0.03900182920501956 + 0.0467973911141459im, -0.02600487162459121 + 0.00677931948613959im, -1.0296381738431903 + 0.0im, -0.06286857560379051 + 0.0522079088192816im, -4.0e-17 - 3.0e-17im  …  0.03677644213926787 - 0.00958740556095872im, -2.0e-17 - 7.0e-17im, -1.1370284928234762 + 0.0im, 0.0444547961329793 - 0.03691656635768264im, -0.03900182920501956 - 0.0467973911141459im, -0.0528966421489845 - 0.11242638179702254im, -0.04533252691316592 - 0.02123122456720352im, 0.01817522823926729 + 0.01812393

In [24]:
#@btime begin
@time begin
basis2 = basis ⊗ basis

# interaction : at_i at_i a_i a_i = at_i a_i at_i a_i - at_i a_i = n_i n_i - n_i
    
Vint = SparseOperator(basis2)

for n in 1:N
    Vint += U/2*transition(basis,n,n)⊗transition(basis,n,n)
end

Vint_sub = (P⊗P)*Vint*(Pt⊗Pt)

Vint_mb = manybodyoperator(basis_mb, Vint_sub)
end

  0.418539 seconds (347.75 k allocations: 23.349 MiB, 99.68% compilation time)


Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)
   0.325738+0.0im         -0.0260049+0.00677932im  …  -0.0390018-0.0467974im
 -0.0260049-0.00677932im    0.303695-0.0im            -0.0528966-0.112426im
  0.0861412-0.0403437im   -0.0628686+0.0522079im      -0.0453325-0.0212312im
   0.118168-0.0109049im      3.0e-17+0.0im             0.0181752+0.0181239im
  0.0122897-0.00227428im  -0.0577122+0.0270292im       0.0444548+0.0369166im
 -0.0390018+0.0467974im   -0.0528966+0.112426im    …    0.379433+0.0im

In [25]:
# executeme

function get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)
    P1 = P.data
    P1t = Pt.data;

    @einsum P4[k,l,m,n] := P1[k,i] * P1[l,i] * P1t[i,m] * P1t[i,n]

    b2_sub = b_sub ⊗ b_sub

    P4re = reshape(P4, N_cut^2, N_cut^2)

    Vint_bsub2 = Operator(b2_sub,U/2*P4re)

    return Vint_bsub2
end

get_Vint_bsub2_einsum (generic function with 1 method)

In [26]:
Vint_bsub2 = get_Vint_bsub2_einsum(P, Pt, b_sub, N_cut)

Operator(dim=9x9)
  basis: [Subspace(superbasis=NLevel(N=9), states:3) ⊗ Subspace(superbasis=NLevel(N=9), states:3)]
    0.162869-1.0e-17im      …  -0.0195009-0.0233987im
 -0.00919411-0.00239685im      -0.0187018-0.0397487im
   0.0304555-0.0142637im       -0.0160275-0.00750637im
 -0.00919411-0.00239685im      -0.0187018-0.0397487im
    0.059084-0.00545245im      0.00908761+0.00906197im
  0.00434506-0.000804078im  …   0.0157171+0.013052im
   0.0304555-0.0142637im       -0.0160275-0.00750637im
  0.00434506-0.000804078im      0.0157171+0.013052im
  -0.0195009+0.0233987im         0.189717-1.0e-17im

In [27]:
Vint_mb_einsum = manybodyoperator(basis_mb, Vint_bsub2)

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)
   0.325738-1.0e-17im     -0.0260049+0.00677932im  …  -0.0390018-0.0467974im
 -0.0260049-0.00677932im    0.303695+0.0im            -0.0528966-0.112426im
  0.0861412-0.0403437im   -0.0628686+0.0522079im      -0.0453325-0.0212312im
   0.118168-0.0109049im      1.0e-17+1.0e-17im         0.0181752+0.0181239im
  0.0122897-0.00227428im  -0.0577122+0.0270292im       0.0444548+0.0369166im
 -0.0390018+0.0467974im   -0.0528966+0.112426im    …    0.379433-2.0e-17im

In [28]:
#@btime begin
@time begin
    Vint_mb_einsum = get_Vint_mb_einsum(P, Pt, b_sub, N_cut, U, basis_mb)
end

LoadError: UndefVarError: `get_Vint_mb_einsum` not defined

In [29]:
isapprox( Vint_mb , Vint_mb_einsum )

true

In [30]:
H_MB = H1_MB + Vint_mb_einsum
H_mb = H1_MB + Vint_mb

Operator(dim=6x6)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=9), states:3), states:6)
   -5.13836+0.0im         -0.0260049+0.00677932im  …  -0.0390018-0.0467974im
 -0.0260049-0.00677932im    -5.16041-2.0e-17im        -0.0528966-0.112426im
  0.0861412-0.0403437im   -0.0628686+0.0522079im      -0.0453325-0.0212312im
   0.118168-0.0109049im     -1.3e-16+2.0e-17im         0.0181752+0.0181239im
  0.0122897-0.00227428im  -0.0577122+0.0270292im       0.0444548+0.0369166im
 -0.0390018+0.0467974im   -0.0528966+0.112426im    …    -5.08467+4.4e-16im

2) If U<<Band-Gap, Sub-Space Hofstadter Finite-U energies converges at Hofstadter Finite-U energies.
3) When all SP states projected to the new sub-space, Sub-Space Hofstadter Finite-U energies have to equal to Hofstadter Finite-U energies.

In [31]:
@nbinclude("Hofstadter MB in Julia.ipynb"; regex=r"#.*executeme")

Hofstadter_Finite_U (generic function with 1 method)

In [32]:
# using Plots

E1 = eigenenergies(dense((H_MB+dagger(H_MB))/2))

E_mb = eigenenergies(dense((H_mb+dagger(H_mb))/2))


E2 = eigenenergies(dense(Hofstadter_Finite_U(Nx, Ny, 1/q, PN, U)))

# print(E1-E2[1:length(E1)])

# plot(1:length(E1),E1,seriestype=:scatter,markershape=:star5, markersize=6, label="Sub-Space")
# plot!(1:length(E2),E2,seriestype=:scatter,label="Real-Space",legend=:topleft)
# xlabel!("n");ylabel!("E")

print(E1,'\n', E_mb,'\n',E2[1:10])

[-5.29743494847109, -5.297434948471089, -5.297434948471085, -4.964101615137757, -4.964101615137756, -4.9641016151377535]
[-5.297434948471091, -5.29743494847109, -5.297434948471087, -4.964101615137757, -4.964101615137755, -4.964101615137754]
[-5.340971176014529, -5.340971176014522, -5.340971176014509, -5.0661682656188365, -5.06616826561883, -5.066168265618821, -1.81285035447887, -1.8128503544788692, -1.8128503544788688, -1.704570336931173]

In [33]:
E_mb

6-element Vector{Float64}:
 -5.297434948471091
 -5.29743494847109
 -5.297434948471087
 -4.964101615137757
 -4.964101615137755
 -4.964101615137754